# Imports

In [1]:
import numpy as np
from scipy.fft import *

In [2]:
np.set_printoptions(edgeitems=30, linewidth=100000, 
    formatter=dict(float=lambda x: "%.3g" % x))

## Setup inputs

In [282]:
N = 4

f_base = np.arange(N)+1
f_grid = np.arange(N**2).reshape(N, N)+1

f_grid

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]])

# Transforms

## 1. DCTs

### 1.1. DCT Type I

In [4]:
f_base_dct1_1d = dct(f_base, 1, norm="ortho", orthogonalize=False)

f_ext_dct1_1d = np.concatenate((f_base, f_base[-2:0:-1]))
res_dct1_1d = (rfft(f_ext_dct1_1d) * 0.5 * np.sqrt(2/(N-1))).real

In [5]:
print("Error: ", np.sum(np.abs(f_base_dct1_1d-res_dct1_1d)))
f_base_dct1_1d, res_dct1_1d

Error:  0.0


(array([6.12, -1.63, -9.06e-17, -0.408]),
 array([6.12, -1.63, -9.06e-17, -0.408]))

In [6]:
f_base_dct1_2d = dctn(f_grid, 1, norm="ortho", orthogonalize=False)

f_ext_dct1_1d = np.zeros((2*N-2, 2*N-2), dtype=np.float64)
f_ext_dct1_1d[:N, :N] = f_grid
f_ext_dct1_1d[N:, :N] = f_grid[-2:0:-1, :]
f_ext_dct1_1d[:N, N:] = f_grid[:, -2:0:-1]
f_ext_dct1_1d[N:, N:] = f_grid[-2:0:-1, -2:0:-1]

res_dct1_2d = (rfft2(f_ext_dct1_1d) * 0.25 * 2/(N-1))[:N, :N].real

In [7]:
print("Error: ", np.sum(np.abs(f_base_dct1_2d-res_dct1_2d)))
f_base_dct1_2d, res_dct1_2d

Error:  8.881784197001252e-16


(array([[51, -4, -2.22e-16, -1],
        [-16, 0, 0, 0],
        [-8.88e-16, 0, 0, 0],
        [-4, 0, 0, 0]]),
 array([[51, -4, -2.22e-16, -1],
        [-16, 0, 0, 0],
        [0, 0, 0, 0],
        [-4, 0, 0, 0]]))

### 1.2. DCT Type II

In [8]:
f_base_dct2_1d = dct(f_base, 2, norm="ortho", orthogonalize=False)

f_ext_dct2_1d = np.zeros(4*N)
f_ext_dct2_1d[1:2*N:2] = f_base
f_ext_dct2_1d[2*N+1:4*N:2] = f_base[::-1]

res_dct2_1d = (rfft(f_ext_dct2_1d) * np.sqrt(2/N) * 0.5)[:N].real

In [9]:
print("Error: ", np.sum(np.abs(f_base_dct2_1d-res_dct2_1d)))
f_base_dct2_1d, res_dct2_1d

Error:  6.106226635438361e-16


(array([7.07, -2.23, 0, -0.159]), array([7.07, -2.23, 0, -0.159]))

In [10]:
f_base_dct2_2d = dctn(f_grid, 2, norm="ortho", orthogonalize=False)

f_ext_dct2_2d = np.zeros((4*N, 4*N))
f_ext_dct2_2d[1:2*N:2, 1:2*N:2] = f_grid
f_ext_dct2_2d[2*N+1:4*N:2, 1:2*N:2] = f_grid[::-1, :]
f_ext_dct2_2d[1:2*N:2, 2*N+1:4*N:2] = f_grid[:, ::-1]
f_ext_dct2_2d[2*N+1:4*N:2, 2*N+1:4*N:2] = f_grid[::-1, ::-1]

res_dct2_2d = (rfft2(f_ext_dct2_2d) * (2/N) * 0.25)[:N, :N].real

In [11]:
print("Error: ", np.sum(np.abs(f_base_dct2_2d-res_dct2_2d)))
f_base_dct2_2d, res_dct2_2d

Error:  1.1102230246251565e-15


(array([[68, -6.31, 0, -0.448],
        [-25.2, 0, 0, 0],
        [0, 0, 0, 0],
        [-1.79, 0, 0, 0]]),
 array([[68, -6.31, 0, -0.448],
        [-25.2, 0, 0, 0],
        [0, 0, 0, 0],
        [-1.79, 0, 0, 0]]))

### 1.3. DCT Type III

In [39]:
f_base_dct3_1d = dct(f_base, 3, norm="ortho", orthogonalize=False)

f_ext_dct3_1d = f_base * 2.0 * np.sqrt(N/2) * np.exp((np.arange(N) * np.pi * 0.5 / N)*1j)

res_dct3_1d = irfft(f_ext_dct3_1d, n=2*N)[:N].real

In [40]:
print("Error: ", np.sum(np.abs(f_base_dct3_1d-res_dct3_1d)))
f_base_dct3_1d, res_dct3_1d

Error:  2.1094237467877974e-15


(array([4.24, -3.22, 0.925, -0.535]), array([4.24, -3.22, 0.925, -0.535]))

In [75]:
f_base_dct3_2d = dctn(f_grid, 3, norm="ortho", orthogonalize=False)

def dct3_1d(inp):

    temp = inp * 2.0 * np.sqrt(N/2) * np.exp((np.arange(N) * np.pi * 0.5 / N)*1j)
    return irfft(temp, n=2*N)[:N].real

f_ext_dct3_2d = np.apply_along_axis(dct3_1d, 0, f_grid)
res_dct3_2d = np.apply_along_axis(dct3_1d, 1, f_ext_dct3_2d)

In [76]:
print("Error: ", np.sum(np.abs(f_base_dct3_2d-res_dct3_2d)))
f_base_dct3_2d, res_dct3_2d

Error:  4.374278717023117e-14


(array([[25.1, -10.9, 3.97, -1.65],
        [-21.4, 7.39, -3.03, 1.04],
        [5.9, -2.22, 0.87, -0.32],
        [-3.6, 1.21, -0.505, 0.168]]),
 array([[25.1, -10.9, 3.97, -1.65],
        [-21.4, 7.39, -3.03, 1.04],
        [5.9, -2.22, 0.87, -0.32],
        [-3.6, 1.21, -0.505, 0.168]]))

## 2. DSTs

### 2.1. DST Type I

In [192]:
f_base_dst1_1d = dst(f_base, 1, norm="ortho", orthogonalize=False)

f_ext_dst1_1d = np.zeros(2*N+2)
f_ext_dst1_1d[1:N+1] = f_base
f_ext_dst1_1d[N+2:] = -f_base[::-1]

res_dst1_1d = -0.5 * fft(f_ext_dst1_1d)[1:N+1].imag * np.sqrt(2 / (N+1))

In [193]:
print("Error: ", np.sum(np.abs(f_base_dst1_1d-res_dst1_1d)))
f_base_dst1_1d, res_dst1_1d

Error:  0.0


(array([4.87, -2.18, 1.15, -0.514]), array([4.87, -2.18, 1.15, -0.514]))

In [225]:
f_base_dst1_2d = dstn(f_grid, 1, norm="ortho", orthogonalize=False)

f_ext_dst1_2d = np.zeros((2*N+2, 2*N+2))
f_ext_dst1_2d[1:N+1, 1:N+1] = f_grid
f_ext_dst1_2d[N+2:, 1:N+1] = -f_grid[::-1, :]
f_ext_dst1_2d[1:N+1, N+2:] = -f_grid[:, ::-1]
f_ext_dst1_2d[N+2:, N+2:] = f_grid[::-1, ::-1]

res_dst1_2d = -0.25 * rfft2(f_ext_dst1_2d)[1:N+1, 1:N+1].real * 2 / (N+1)

In [226]:
print("Error: ", np.sum(np.abs(f_base_dst1_2d-res_dst1_2d)))
f_base_dst1_2d, res_dst1_2d

Error:  1.3716303691971395e-13


(array([[101, -9.1, 29, -3.49, 11.2, -1],
        [-54.6, -6.66e-15, -15.6, -9.77e-15, -6, 3.55e-15],
        [29, -2.6, 8.31, -1, 3.19, -0.286],
        [-21, -1.78e-15, -6, 4.44e-16, -2.3, -0],
        [11.2, -1, 3.19, -0.384, 1.23, -0.11],
        [-6, -0, -1.72, -1.11e-16, -0.659, 4.44e-16]]),
 array([[101, -9.1, 29, -3.49, 11.2, -1],
        [-54.6, -1.93e-15, -15.6, -1.24e-15, -6, -3.77e-15],
        [29, -2.6, 8.31, -1, 3.19, -0.286],
        [-21, -2.23e-16, -6, 5.51e-16, -2.3, -3.02e-15],
        [11.2, -1, 3.19, -0.384, 1.23, -0.11],
        [-6, -1.14e-15, -1.72, 9.92e-16, -0.659, -7.48e-16]]))

### 2.2. DST Type II

In [237]:
f_base_dst2_1d = dst(f_base, 2, norm="ortho", orthogonalize=False)

f_ext_dst2_1d = f_base
f_ext_dst2_1d[1::2] = -f_base[1::2]

res_dst2_1d = dct(f_ext_dst2_1d, 2, norm="ortho", orthogonalize=False)[::-1]

In [238]:
print("Error: ", np.sum(np.abs(f_base_dst2_1d-res_dst2_1d)))
f_base_dst2_1d, res_dst2_1d

Error:  0.0


(array([0.0801, 0, 0.408, 0, 4.16, -12.1]),
 array([0.0801, 0, 0.408, 0, 4.16, -12.1]))

In [239]:
f_base_dst2_2d = dstn(f_grid, 2, norm="ortho", orthogonalize=False)

f_ext_dst2_2d = f_grid
f_ext_dst2_2d[1::2, ::2] = -f_grid[1::2, ::2]
f_ext_dst2_2d[::2, 1::2] = -f_grid[::2, 1::2]

res_dst2_2d = dctn(f_ext_dst2_2d, 2, norm="ortho", orthogonalize=False)[::-1, ::-1]

In [240]:
print("Error: ", np.sum(np.abs(f_base_dst2_2d-res_dst2_2d)))
f_base_dst2_2d, res_dst2_2d

Error:  0.0


(array([[92.1, -7.73, 33.7, -4.46, 24.7, -3.86],
        [-46.4, 0, -17, 0, -12.4, 0],
        [33.7, -2.83, 12.3, -1.63, 9.03, -1.41],
        [-26.8, 0, -9.8, 0, -7.17, 0],
        [24.7, -2.07, 9.03, -1.2, 6.61, -1.04],
        [-23.2, 0, -8.49, 0, -6.21, 0]]),
 array([[92.1, -7.73, 33.7, -4.46, 24.7, -3.86],
        [-46.4, 0, -17, 0, -12.4, 0],
        [33.7, -2.83, 12.3, -1.63, 9.03, -1.41],
        [-26.8, 0, -9.8, 0, -7.17, 0],
        [24.7, -2.07, 9.03, -1.2, 6.61, -1.04],
        [-23.2, 0, -8.49, 0, -6.21, 0]]))

### 2.3. DST Type III

In [258]:
f_base_dst3_1d = dst(f_base, 3, norm="ortho", orthogonalize=False)

f_ext_dst3_1d = f_base[::-1]
coeff = np.ones(N)
coeff[1::2] = -1

res_dst3_1d = dct(f_ext_dst3_1d, 3, norm="ortho", orthogonalize=False) * coeff

In [259]:
print("Error: ", np.sum(np.abs(f_base_dst3_1d-res_dst3_1d)))
f_base_dst3_1d, res_dst3_1d

Error:  0.0


(array([2.08, -1.39, -0.288, -0.907, -0.239, -6.54]),
 array([2.08, -1.39, -0.288, -0.907, -0.239, -6.54]))

In [279]:
f_base_dst3_2d = dstn(f_grid, 3, norm="ortho", orthogonalize=False)

f_ext_dst3_2d = f_grid[::-1, ::-1]
coeff = -np.ones((N, N))
coeff[1::2, 1::2] = 1
coeff[::2, ::2] = 1

res_dst3_2d = dctn(f_ext_dst3_2d, 3, norm="ortho", orthogonalize=False) * coeff

In [280]:
print("Error: ", np.sum(np.abs(f_base_dst3_2d-res_dst3_2d)))
f_base_dst3_2d, res_dst3_2d

Error:  0.0


(array([[-0.0477, -0.126, -0.254, -0.402, -0.81, -2.11],
        [-0.00626, 0.0558, 0.0384, 0.158, 0.0893, 1.66],
        [-0.135, -0.286, -0.65, -0.931, -2.1, -4.05],
        [-0.0522, 0.0731, -0.0686, 0.176, -0.315, 3.36],
        [-0.486, -1.09, -2.4, -3.53, -7.72, -16.2],
        [1.11, 4.88, 7.84, 15, 24.1, 101]]),
 array([[-0.0477, -0.126, -0.254, -0.402, -0.81, -2.11],
        [-0.00626, 0.0558, 0.0384, 0.158, 0.0893, 1.66],
        [-0.135, -0.286, -0.65, -0.931, -2.1, -4.05],
        [-0.0522, 0.0731, -0.0686, 0.176, -0.315, 3.36],
        [-0.486, -1.09, -2.4, -3.53, -7.72, -16.2],
        [1.11, 4.88, 7.84, 15, 24.1, 101]]))